In [1]:
import polars as pl
import pandas as pd
from sklearn.model_selection import train_test_split
import catboost
import numpy as np
from sklearn.metrics import roc_auc_score

In [2]:
def dataframe_read(path):
    temp=pd.read_parquet(path)
    display(temp.info(memory_usage="deep"))
    #temp = temp.apply(pd.to_numeric, downcast='float')
    #temp = temp.apply(pd.to_numeric, downcast='integer')
    #display(temp.info(memory_usage="deep"))
    return temp
list_item= dataframe_read("train_interactions.parquet").query("like==1 or dislike==1")['item_id'].unique()

train = dataframe_read("train_interactions.parquet").query("item_id in @list_item")
display(train.info(memory_usage="deep"))
train["like"]= train["like"].astype(int)
train["dislike"]= train["dislike"].astype(int)
train['weight']= train["like"]-train["dislike"]
train = train.drop(columns=['timespent', 'like', 'dislike', 'share',
       'bookmarks'],axis=1)
items =  dataframe_read("items_meta.parquet")
users=  dataframe_read("users_meta.parquet")


display(train.info(memory_usage="deep"))


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 145667282 entries, 0 to 145667281
Data columns (total 7 columns):
 #   Column     Dtype 
---  ------     ----- 
 0   user_id    uint32
 1   item_id    uint32
 2   timespent  uint8 
 3   like       uint8 
 4   dislike    uint8 
 5   share      uint8 
 6   bookmarks  uint8 
dtypes: uint32(2), uint8(5)
memory usage: 1.8 GB


None

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 145667282 entries, 0 to 145667281
Data columns (total 7 columns):
 #   Column     Dtype 
---  ------     ----- 
 0   user_id    uint32
 1   item_id    uint32
 2   timespent  uint8 
 3   like       uint8 
 4   dislike    uint8 
 5   share      uint8 
 6   bookmarks  uint8 
dtypes: uint32(2), uint8(5)
memory usage: 1.8 GB


None

<class 'pandas.core.frame.DataFrame'>
Index: 142037246 entries, 0 to 145667281
Data columns (total 7 columns):
 #   Column     Dtype 
---  ------     ----- 
 0   user_id    uint32
 1   item_id    uint32
 2   timespent  uint8 
 3   like       uint8 
 4   dislike    uint8 
 5   share      uint8 
 6   bookmarks  uint8 
dtypes: uint32(2), uint8(5)
memory usage: 2.8 GB


...

None

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 183404 entries, 0 to 183403
Data columns (total 3 columns):
 #   Column   Non-Null Count   Dtype 
---  ------   --------------   ----- 
 0   user_id  183404 non-null  uint32
 1   gender   183404 non-null  uint8 
 2   age      183404 non-null  uint8 
dtypes: uint32(1), uint8(2)
memory usage: 1.0 MB


None

<class 'pandas.core.frame.DataFrame'>
Index: 142037246 entries, 0 to 145667281
Data columns (total 3 columns):
 #   Column   Dtype 
---  ------   ----- 
 0   user_id  uint32
 1   item_id  uint32
 2   weight   int64 
dtypes: int64(1), uint32(2)
memory usage: 3.2 GB


None

In [3]:
users['age_category'] =(round(users['age']/10,0)*10).astype(str)
data=train.merge(users,how='left',on='user_id').merge(items[['item_id', 'source_id', 'duration']],how='left',on='item_id')
%xdel train
train_item=data[['user_id','item_id', 'weight', 'gender', 'age_category','source_id']].drop_duplicates()

In [4]:
pivot_item=pd.pivot_table(data=data,index=['item_id','source_id','gender', 'age_category'],columns='weight',values=['user_id'], aggfunc='count').fillna(0).reset_index()
pivot_item.columns=['item_id','source_id','gender', 'age_category','dislike','like']
%xdel data
train_item=train_item.merge(pivot_item,on=['item_id', 'gender', 'age_category','source_id'],how='left')

ValueError: Length mismatch: Expected axis has 7 elements, new values have 6 elements

In [ ]:
test_pairs = pd.read_csv('test_pairs.csv')
test_item=test_pairs.merge(users,how='left',on='user_id').merge(items[['item_id', 'source_id']],how='left',on='item_id').drop(columns='age').merge(pivot_item,on=['item_id', 'gender', 'age_category','source_id'],how='left')

In [ ]:
%xdel users
%xdel items

ind_cat_cols=['gender', 'age_category','item_id','source_id']

boosting_model =  catboost.CatBoostClassifier(iterations=1, depth=8, cat_features=ind_cat_cols,
                           custom_metric=['AUC'], random_seed=42)
param_grid = {'learning_rate': np.linspace(0, 1, 11),'l2_leaf_reg': np.linspace(0, 1, 11)}
boosting_model.grid_search(param_grid, 
                           train_final.drop(columns=['user_id','weight'],axis=1), 
                           train_final['weight'], plot=True, refit=True)

In [ ]:
ind_cat_cols=['gender', 'age_category','item_id','source_id']
x_cols=['item_id','gender','age_category','source_id','dislike','like']
y_col=['weight']
boosting_model =  catboost.CatBoostClassifier(n_estimators=10, silent=True, cat_features=ind_cat_cols, eval_metric='AUC')
param_grid = { 'learning_rate': np.linspace(0, 1, 11),'l2_leaf_reg': np.linspace(0, 1, 11)             }
boosting_model.grid_search(param_grid, 
                           train_item[x_cols], 
                           train_item[y_col], plot=True, refit=True)

In [ ]:
x_cols=['item_id','gender','age_category','source_id','dislike','like']

boosting_model = catboost.CatBoostClassifier(n_estimators=10, silent=True, cat_features=ind_cat_cols, eval_metric='AUC',learning_rate=1, l2_leaf_reg=0.4)
boosting_model.fit(train_item[x_cols],train_item[y_col], cat_features=ind_cat_cols)
y_train_predicted = boosting_model.predict_proba(train_item[x_cols])[:, 1]
y_train_pred = boosting_model.predict(train_item[x_cols])
y_test_proba = boosting_model.predict_proba(test_item[x_cols])[:, 1]
y_test_pred = boosting_model.predict(test_item[x_cols])

In [ ]:
test_pairs['predict']=y_test_proba
test_pairs.to_csv('sample_submission.csv', index=False)